# 3-Exploring the Data
In this notebook we start exploring the database downloaded and created by the previous notebooks.

**Requirements:**

- Please run the `02-import-data.ipynb` notebook first so that you create the SQLite database.
- Recommended install: [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/user_install.html)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from db.api import VedDb

Start by creating the `db` variable of type `VedDb`. This is our API to the database. All the queries and modifications will happen through this object.

In [ ]:
db = VedDb()

The `VedDb` class contains a number of convenience functions to help you explore the database through Pandas DataFrames. The `query_df` function executes an arbitrary SQL query against the database and stores the result on a Pandas DataFrame for further inspection or analysis. The sister function `query` does the same thing but stores the result on a list of tuples instead.

The following command reads all the vehicles into a DataFrame:

In [ ]:
vehicles = db.query_df("select * from vehicle")

We can now run a Pandas query on that DataFrame:

In [ ]:
vehicles.groupby(by='vehicle_type').count()

If you want to display the result of a query, you can just display the resulting DataFrame contents, like so:

In [ ]:
db.query_df("select * from vehicle").head(10)

To make it a bit more convenient, you can also use the `head` and `tail` functions:

In [ ]:
db.head("select * from signal where vehicle_id = 10")

In [ ]:
db.tail("select * from signal where vehicle_id = 10")

If all you need is a scalar, like when counting records, there's another function for you: `query_scalar`. Here I illustrate its use to highlight the difference between the count of distinct `trip_id` values and distinct pairs of `vehicle_id` and `day_num`, the real way to determine trips.

In [ ]:
sql = """
select count(0) from (
    select distinct vehicle_id, day_num
    from signal 
) as distinct_trips
"""
db.query_scalar(sql)

Now we get the number of different instances of `trip_id`.

In [ ]:
db.query_scalar("select count(distinct trip_id) from signal")

Finally, let's plot an histogram of trip frequencies, using data previously stored on the `move` table:

In [ ]:
sql = """
select    vehicle_id
,         count(day_num) as num_trips
from      move
group by  vehicle_id"""
trip_counts = db.query_df(sql)

In [ ]:
hh = plt.hist(trip_counts['num_trips'].values, bins=50)